<a href="https://colab.research.google.com/github/AhmedMahmood19/Topic-Modeling-IMDB-Reviews/blob/main/Topic_Modeling_on_IMDB_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
#**Don't run the sections Data Collection, and Data Cleaning!**  
Instead you can download the cleaned dataset and save time.
1.   Run the cell below with the gdown command
2.   Run the cell in the Import required libraries section
3.   Start running the cells directly from the Data Exploration section

**Alternatively, you can view the output from the previously run cells in the notebook, since the models take a while to run.**

---



In [ ]:
!pip install gdown
!gdown --id 1t3RSPIKArU5yFnNjhi1n2Ip-efKXRiZV

#Import required libraries

In [ ]:
!pip install pyLDAvis
!pip install gensim
from gensim.corpora import Dictionary
from gensim.models import LsiModel, Phrases, LdaModel, TfidfModel
from gensim.utils import simple_preprocess
import pyLDAvis.lda_model
import os
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

import spacy
nlp = spacy.load("en_core_web_sm")

pd.set_option('display.max_colwidth', 400)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Data Collection

We will first download the dataset called "Large Movie Review Dataset v1.0" from https://ai.stanford.edu/~amaas/data/sentiment/

We will be using the additional 50,000 unlabeled documents the dataset contains for unsupervised learning in the directory "train/unsup/". 

This dataset contains movie reviews from IMDB of a variety of movies, there are no more than 30 reviews for any given movie.

Download the folder containing the dataset and extract the files from the downloaded folder

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzvf "/content/aclImdb_v1.tar.gz" -C "/content"

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/4

Convert the dataset into a dataframe with a single column called 'document', each row represents a single review from the unsupervised dataset

In [ ]:
directory_path = '/content/aclImdb/train/unsup'

doclist = []
#Iterate through the filenames in the unsup directory
for filename in os.listdir(directory_path):
    #Append filename to the directory path and open the file
    with open(os.path.join(directory_path, filename), 'r') as file:
        #Read the text file and append its contents to the list 
        doc = file.read()
        doclist.append(doc)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(doclist, columns=['document'])
df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,document
0,"This was a ridiculous version of male hustlers and other assorted homeless teens because it is done in such a clean-cut manner. I assume that the people that drifter teen Johnny encounter during his stay in Las Vegas would be much more hostile, crazy, and the like. If you're not looking for a vigorous story of similar circumstances like say, 'Where the Day Takes You,' then Speedway Junky is a ..."
1,"I strongly don't recommend this movie unless your a fan of the 70's porno genre.It is quite tame by today's standard,but the plot is just as incoherent as the modern porn plot.Barbara Broadcast is a author who takes tricks on the side because its so much fun!She is not that bad looking of a lady,kind of like a Julieann Moore type.Only thing is she is not nearly as attractive.Attractiveness is ..."
2,"What is it with film directors that they cannot produce consistently good work once fame and reverence goes to their head? Scorsese is considered the greatest US filmmaker still active (by all those who don't think so of Spielberg). Much like his endlessly-feuding, barbaric, mid-19th century New Yorkers, the man seems to have worn his welcome in history, and what a sad, sad realization that is..."
3,"It's hard to express sometimes just how bad a motion picture is. I mean after all, haven't we ALL had someone come up to us and say, ""It's the WORST motion picture I've ever seen?"" After a while, the term really doesn't have much meaning anymore.<br /><br />While I can't quite say that this is the worst horror film I've ever seen (after seeing literally thousands of them, that honor I give to ..."
4,"I have to be honest before I give my opinion towards this movie. I was looking for Shannon Tweed's ""Singapore Sling"" (1998) but by accident, I got this movie. I don't regret. Strangely enough, I ended up liking it. It's a raw movie that goes directly to the point. Also, I couldn't believe what I was watching. I wasn't exactly shocked by it, but I was watching in awe and incredulity. It's just ..."


#Data Cleaning

Remove any duplicate reviews

In [ ]:
print("Unique values:",len(df['document'].unique()),", Actual number of rows:", df.shape[0])
df=df.drop_duplicates().reset_index(drop=True)
print("Dropped duplicates\nUnique values:",len(df['document'].unique()),", Actual number of rows:", df.shape[0])

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Unique values: 49507 , Actual number of rows: 50000
Dropped duplicates
Unique values: 49507 , Actual number of rows: 49507


Lemmatization generally results in better coherence of topics and improves topic interpretability by reducing words to their base form, which helps to group together words with similar meanings and also reduces redundancy.

I observed that there were many single/double letter words present in the reviews that were not actual words and were not useful in interpreting topics so I decided to remove them.

Characters that aren't part of the alphabet don't help with topic interpretability so I removed them.



In [ ]:
def clean(doc):
    # Make all letters lowercase
    doc=doc.lower()
    # remove the line break tag that appears in almost every review
    doc = re.sub(r'<br \/><br \/>', ' ', doc)
    # remove any punctuation marks, special characters and digits 
    doc = re.sub(r'[^a-zA-Z\s]', ' ', doc)
    
    doc = nlp(doc)
    # remove stopwords and lemmatize the tokens
    filtered_tokens = [token.lemma_ for token in doc if not token.is_stop]
    # join the filtered tokens back into a document/string
    filtered_document = ' '.join(filtered_tokens)
    # remove single/double lettered words
    filtered_document = re.sub(r'\b\w{1,2}\b', '', filtered_document)
    # remove any excess whitespace
    filtered_document = re.sub(r'\s+', ' ', filtered_document).strip()
    return filtered_document

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Apply the clean function on every document
df['document'] = df['document'].apply(clean)
# Save the cleaned dataset
df.to_csv("IMDB-Reviews-clean.csv", index=False)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Data Exploration

##This will be used to create a document-term matrix from the dataset of reviews(documents)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def doc_term_matrix(df):
    cv = CountVectorizer()

    # Form a document term matrix using the document column of df
    # dtm is a sparse matrix containing elements in the form: (row,col)->frequency
    dtm = cv.fit_transform(df['document'])

    # Print some info about our document-term matrix
    print("Total no. of documents:",dtm.shape[0])
    print("\nSize of the vocabulary:",dtm.shape[1],"\n")
    return cv,dtm

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Further cleaning
I sorted the terms by their total frequency in the corpus and analyzed the words at the top of the list to determine if they helped to distinguish topics.  
I found several words that were common in all the reviews, such as 'movie', 'film', 'like', 'good', 'bad', 'great', 'way', that did not help to distinguish topics.  
As a result, I defined these terms in the set "custom_stopwords"
and removed these from the corpus

In [ ]:
'''
Sort the terms by the sum of their frequencies across the corpus

For topic modeling, it's recommended to use the sum instead of mean.
Since the mean may result in high values for terms common across many documents
and thus not help distinguish topics.
The sum may give high values to terms that are only common in some documents and
thus help distinguish topics.
'''

df_clean = pd.read_csv("IMDB-Reviews-clean.csv")
cv_clean,dtm_clean = doc_term_matrix(df_clean)

# This is a list of the sum of freqs
sums = dtm_clean.sum(axis=0).tolist()[0]
# This is a list of the corresponding terms
terms = cv_clean.get_feature_names_out().tolist()
# Create a dataframe of terms and their frequency sum then sort it by the sum
sumdf = pd.DataFrame({'Term': terms, 'Freq Sum': sums})
sumdf.sort_values(by='Freq Sum', ascending=False, inplace=True)
sumdf.head(10)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total no. of documents: 49507

Size of the vocabulary: 83708 



,Term,Freq Sum
49118,movie,102153
25896,film,96380
42587,like,44275
29994,good,40774
74275,time,30904
12010,character,28722
80505,watch,26860
4956,bad,26534
70412,story,25565
65055,see,23885


In [ ]:
custom_stopwords = {'scene', 'time', 'story', 'character', 'typical', 'clearly', 'cool', 'have', 'particularly', 'guess', 'amazing', 'fine', 'wonderful', 'note', 'despite', 'instead', 'movie', 'explain', 'well', 'let', 'wait', 'know', 'think', 'bring', 'somewhat', 'james', 'good', 'right', 'give', 'happen', 'ago', 'usually', 'see', 'simply', 'exactly', 'sure', 'way', 'totally', 'come', 'get', 'obviously', 'probably', 'like', 'truly', 'usual', 'able', 'example', 'great', 'stuff', 'david', 'absolutely', 'provide', 'etc', 'keep', 'excellent', 'actually', 'entire', 'robert', 'realize', 'look', 'suppose', 'richard', 'obvious', 'kind', 'situation', 'bad', 'say', 'ask', 'use', 'nice', 'important', 'overall', 'film', 'unfortunately', 'feel', 'reason', 'thank', 'one', 'soon', 'especially', 'take', 'york', 'lot', 'apparently', 'michael', 'john', 'yes', 'maybe', 'deserve', 'want', 'watch', 'definitely', 'basically', 'perfect', 'expect', 'brilliant', 'highly', 'decide', 'thing', 'possible', 'extremely', 'self', 'consider', 'certainly', 'will', 'opinion', 'non', 'jack', 'play', 'people', 'don', 'make', 'doesn', 'would', 'try', 'didn'}

def remove_custom_stopwords(document, custom_stopwords):
    tokens = word_tokenize(document)
    filtered_tokens = [t for t in tokens if t not in custom_stopwords]
    filtered_document = ' '.join(filtered_tokens)
    return filtered_document

df_clean['document'] = df_clean['document'].apply(remove_custom_stopwords, custom_stopwords=custom_stopwords)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


I sorted the terms again by their total frequency in the corpus and we can see that the words at the top of the list are now more helpful to distinguish topics.

In [ ]:
cv_clean,dtm_clean = doc_term_matrix(df_clean)

# This is a list of the sum of freqs
sums = dtm_clean.sum(axis=0).tolist()[0]
# This is a list of the corresponding terms
terms = cv_clean.get_feature_names_out().tolist()
# Create a dataframe of terms and their frequency sum then sort it by the sum
sumdf = pd.DataFrame({'Term': terms, 'Freq Sum': sums})
sumdf.sort_values(by='Freq Sum', ascending=False, inplace=True)
sumdf.head(10)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total no. of documents: 49507

Size of the vocabulary: 83591 



,Term,Freq Sum
43460,love,16866
25963,find,16129
44744,man,15572
22977,end,15180
42456,life,14680
56288,plot,13913
81977,work,13551
586,actor,13212
82605,year,13137
42849,little,12725


We observed that, the top 5454 most frequent terms account for a significant portion of the total word frequency.  
Therefore, to improve the quality and efficiency of topic modeling, we will only consider terms that have a total frequency greater than 100, as they are more likely to be meaningful and informative.

In [ ]:
sumdf[5454:]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Term,Freq Sum
48017,misunderstanding,100
82672,yep,100
22702,embarrassingly,100
78721,vapid,100
1573,alec,100
...,...,...
39802,kilroy,1
39804,kiltathi,1
39805,kilte,1
39809,kimber,1


#Topic Modeling

##**LDA (Latent Dirichlet Allocation)**  
We can assume that a **document** consists of a probability distribution **(a mix) of topics** and a **topic** consists of a probability distribution **(a mix) of terms.** We want LDA to learn these probability distributions.

LDA requires a term-document matrix, the no. of topics and the no. of iterations to perform.

*   First LDA starts by randomly assigning a topic to each word in each document.
*   Then it goes through all of the words again and judges if the topic for a word needs to be changed.  
* It repeats the above step for the no. of iterations it was told to perform.
* The user then looks at the distributions to judge if the topics discovered look accurate and distinct and they may name these topics.

*For example, initially word X was assigned topic A in document 1.*

LDA evaluates if the topic for that word needs to be reassigned using 2 criteria:
1.   Does topic A infrequently occur in document 1? 
2.   Does word X infrequently occur in topic A across all documents?




---
##Using scikit-learn for LDA 

###1st Attempt:
Now that we have removed any domain-specific stopwords, let's attempt to generate topics from the corpus and see how coherent and distinct they are on our first attempt.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

#max_df means that if any term appears in more than 95% of the documents then it will be excluded
#min_df means that if any term appears in less than 10 documents then it will be excluded
vec = CountVectorizer(max_df=0.95, min_df=2, max_features=5454)
dtm = vec.fit_transform(df_clean['document'])
feature_names = vec.get_feature_names_out()

# n_components is the no. of topics
# learning_method is the method for updating the topic probabilities during each iteration
# "online" learning method allows for faster and more memory-efficient training
# learning_offset downweights early iterations and helps the model converge faster
# random_state ensures reproducibility of the results, essential for experimenting
lda = LatentDirichletAllocation(n_components=10, max_iter=10, learning_method='online', learning_offset=50.,random_state=0).fit(dtm)

#how many of the top words for each topic to print
no_top_words = 10
for topic_no, topic in enumerate(lda.components_):
        print("Topic", topic_no+1, ":")
        top_words = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
        print(" ".join(top_words))

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic 1 :
action war fight kill man hero gun game guy cop
Topic 2 :
role performance star cast actor man comedy work support lead
Topic 3 :
guy stupid kid money waste acting video rent end plot
Topic 4 :
life love man find end real world woman child family
Topic 5 :
american black documentary white history director style footage western political
Topic 6 :
horror kill killer dead girl house end murder plot death
Topic 7 :
man year version woman young love old novel set book
Topic 8 :
song music love girl dance musical old voice rock year
Topic 9 :
plot actor script acting director work book cast performance act
Topic 10 :
funny series comedy episode laugh find love joke year original


Here are some possible interpretations of what the topics above could be:  
Topic 1 Action movies.  
Topic 2 Evaluation of performances and roles of actors.  
Topic 3 Movies considered a waste of money due to their poor acting and plot.  
Topic 4 Romantic and family movies.  
Topic 5 Documentaries about american history and politics that might discuss race.  
Topic 6 Horror movies.  
Topic 7 Film adaptations of novels.  
Topic 8 Musical films.  
Topic 9 Evaluation of plot, script, cast, and directors.  
Topic 10 Comedy TV shows.

---
The visualisation below shows that the LDA model worked well and has successfully identified meaningful topics in the data since the circles that represent the topics are not overlapping much, which suggests that the topics are distinct and well-separated from each other.

You can hover over or click any of these circles to see which words they are comprised of.  
(Note: The topic numbers printed above won't correspond to the topic numbers in the graph)

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.lda_model.prepare(lda, dtm, vec)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.012086 -0.014935       1        1  21.488746
8     -0.037878 -0.168589       2        1  13.379664
5     -0.022094 -0.091466       3        1  11.039039
2     -0.172817 -0.122289       4        1  10.684041
1     -0.025873  0.112237       5        1   9.247933
0      0.087313 -0.048309       6        1   8.381428
9     -0.126587 -0.043621       7        1   8.104057
6      0.103987  0.143002       8        1   7.360891
4      0.316865 -0.024440       9        1   6.036042
7     -0.135002  0.258410      10        1   4.278161, topic_info=         Term          Freq         Total Category  logprob  loglift
2340   horror   7660.000000   7660.000000  Default  30.0000  30.0000
1976    funny   9128.000000   9128.000000  Default  29.0000  29.0000
2928     love  16848.000000  16848.000000  Default  28.0000  28.0000
49     action   8274.000000   8274.000000  Default  27.0000  27.0000
937    comedy   7521.000000   7521.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1231      day    966.866465   7855.238239  Topic10  -5.0134   1.0568
3319      new    971.163389   8765.865101  Topic10  -5.0089   0.9515
2884   little    789.217740  12644.852264  Topic10  -5.2164   0.3777
859   classic    671.667939   3956.051203  Topic10  -5.3777   1.3784
2232     hear    660.189554   4125.502892  Topic10  -5.3949   1.3192

[772 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
19        1  0.960311    abuse
19        5  0.038932    abuse
22        2  0.068064  academy
22        5  0.928875  academy
47        1  0.202834      act
...     ...       ...      ...
5444      8  0.192639    young
5444      9  0.000485    young
5444     10  0.053255    young
5451      3  0.911126   zombie
5451      4  0.088417   zombie

[2209 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 6, 3, 2, 1, 10, 7, 5, 8])

###2nd Attempt: 
To improve the quality of the generated topics, we will extract the nouns and adjectives from the corpus. 

Extracting parts of speech, specifically nouns and adjectives, can be helpful for topic modeling because they often carry important semantic information about the topics being discussed in the text.  
By focusing on these parts of speech, we can potentially improve the coherence and interpretability of our topic models, as they are likely to capture the most salient features of the text.

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def extract_parts_of_speech(document):
    doc = nlp(document)
    filtered_tokens = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ']]
    filtered_document = ' '.join(filtered_tokens)
    return filtered_document

df_pos = df_clean.copy()
df_pos['document']=df_pos['document'].apply(extract_parts_of_speech)
df_pos.head(10)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,document
0,actor shannyn likable willing helpful role mystery dead picture jaded detective retire case case connection new cop work case narrative difficult follow flashback contradict motivation viewer compelling complex rich examine chase ghost confidential
1,waste actor money sense care judge point man irresponsible crass cad importance love family trite line lead predictable unrealistic mention multiple line wrap elaine interested fake french accent guy earth strong reaction potential publisher street couple reconciliation meat final theater sheer nonsense view disappointing line trailer cute shandling
2,period broad sweeping comic book gesture dog hero guy french horn theme remind music young classic romp tale little bonus book interest titular support folk heavy fascinating tale interesting actor interesting rock
3,year old aspire journalist upper middle class suburb connection publisher major newspaper publish article interesting plot different direction turn article plot interesting adult publish article sensation young friend year old gopher brother beginning attempt publisher worker newspaper end fall grace final article team member community replete correct anachronistic idea hobo today immigrant fa...
4,naive childish mistake cheesiness valid absurd understand budget opportunity detail ashamed easy space money orphanage blessing unfortunate kid hatred comprehend unsuccessful sequel naive problematic fundamental element main actor antagonist protagonist stupid joke acting series turkish channel inner peace
5,animation cgi big name star dollar budget pure simple genius exception age remind life council estate kid flat estate appearance inhabitant drunk family quiet family family family odd kid angry violent boyfriend single mum daughter life estate wonder life family reality funny painful dull predictable different level
6,fall voice year middle season spongebob change humor change spongebob new smile freak original smile plot new episode pointless spongebob stupid studio head high rating end spongebob early year
7,banana age trainwreck soft porn cinemax air late night audacious cut edge silly boring part funny follow potential green light shine drac face blood pathetic daughter funny sex boring woman vague boorish waste total pretentious claptrap
8,quality bollywood stunt slow repetitive emotional happy young man return day simple magical behavior river silent stream kapoor natural kapoor art direction cinematography credit department beautiful interested music background song monty director dance sequence force perform hall laugh masala
9,masterpiece emotion role fit role proud stand truth effort passion doubt recent achievement speak admiration accomplishment heartiest congratulation success


In [ ]:
vec_pos = CountVectorizer(max_df=0.95, min_df=2, max_features=5454)
dtm_pos = vec_pos.fit_transform(df_pos['document'])
feature_names_pos = vec_pos.get_feature_names_out()
lda_pos = LatentDirichletAllocation(n_components=10, max_iter=10, learning_method='online', learning_offset=50.,random_state=0).fit(dtm_pos)

no_top_words = 10
for topic_no, topic in enumerate(lda_pos.components_):
        print("Topic", topic_no+1, ":")
        top_words = [feature_names_pos[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
        print(" ".join(top_words))

Topic 1 :
book game series original episode version love novel year animation
Topic 2 :
actor plot action director performance work interesting role little real
Topic 3 :
life world man human work city place new question power
Topic 4 :
war documentary man black history fact issue german ship soldier
Topic 5 :
music song performance production actor year work musical role screen
Topic 6 :
girl guy school car high boy cop end police action
Topic 7 :
love old year life comedy family child young man woman
Topic 8 :
horror murder monster plot killer thriller effect dead scary budget
Topic 9 :
funny plot minute actor stupid terrible awful line script waste
Topic 10 :
man woman sex violence evil dead guy blood action western


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Some of the topics that can be interpreted from above may be:  
Topic 1: Entertainment media  
Topic 2: Acting and filmmaking  
Topic 3: Life and society  
Topic 4: War and history  
Topic 5: Musicals  
Topic 6: Highschool dramas with crime  
Topic 7: Romantic and family movies  
Topic 8: Horror movies  
Topic 9: Criticism of comedy movies  
Topic 10: Violent action movies  

---
The visualisation below shows that using the nouns and adjectives may have worked better since the circles representing the topics are overlapping even less than the previous attempt.

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.lda_model.prepare(lda_pos, dtm_pos, vec_pos)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.110148  0.063484       1        1  17.544912
6     -0.014811  0.035175       2        1  13.257819
8     -0.192247 -0.088665       3        1  12.525642
2      0.148014  0.077588       4        1  11.539061
5      0.031661 -0.084049       5        1   8.695848
7     -0.087362 -0.134401       6        1   8.377866
4     -0.153953  0.095419       7        1   7.595213
9      0.174550 -0.274828       8        1   7.030182
3      0.234872  0.158419       9        1   6.907413
0     -0.030576  0.151859      10        1   6.526044, topic_info=         Term          Freq         Total Category  logprob  loglift
2364   horror   7345.000000   7345.000000  Default  30.0000  30.0000
2921      man  13639.000000  13639.000000  Default  29.0000  29.0000
501      book   4785.000000   4785.000000  Default  28.0000  28.0000
3163    music   5858.000000   5858.000000  Default  27.0000  27.0000
2082     girl   7691.000000   7691.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
3952  release    603.909878   2924.933573  Topic10  -5.5074   1.1518
4244   second    613.197938   3993.913660  Topic10  -5.4921   0.8555
1771     fact    653.318064   7357.950482  Topic10  -5.4287   0.3079
1601      end    669.862277   9384.567492  Topic10  -5.4037   0.0896
2859     long    574.188974   5046.682334  Topic10  -5.5578   0.5558

[725 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
43        1  0.680585  action
43        3  0.001371  action
43        5  0.167373  action
43        8  0.150673  action
47        1  0.476680   actor
...     ...       ...     ...
5445      7  0.007567   young
5445      8  0.050396   young
5445      9  0.084374   young
5445     10  0.018845   young
5451      8  0.999192  zombie

[1896 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 9, 3, 6, 8, 5, 10, 4, 1])

---
## Using Gensim for LDA

### Preparing the Gensim Corpus

Tokenizing the Data first, because gensim requires a list of tokens (strings), and preparing it in the "Corpus" and "Dictionary" format which all gensim-models use.

In [ ]:
tokenized_data = df_clean['document'].apply(lambda x: simple_preprocess(x))

dictionary = Dictionary(tokenized_data)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Creating a LDA Model

In [ ]:
# Setting random_state to 0 for reproducibility
# ldamodel = LdaModel(corpus=corpus, num_topics=7, id2word=dictionary, iterations=50, offset=50., random_state = 0)
ldamodel = LdaModel(corpus=corpus, id2word=dictionary, num_topics=8, chunksize=2000, passes=10, iterations=50, alpha='auto', eta='auto', random_state=42)
ldamodel.show_topics()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.017*"role" + 0.015*"book" + 0.014*"performance" + 0.010*"star" + 0.009*"cast" + 0.009*"novel" + 0.007*"version" + 0.005*"king" + 0.005*"love" + 0.005*"actor"'),
 (1,
  '0.009*"funny" + 0.009*"actor" + 0.008*"guy" + 0.008*"acting" + 0.007*"find" + 0.007*"comedy" + 0.007*"end" + 0.006*"laugh" + 0.006*"act" + 0.006*"love"'),
 (2,
  '0.044*"war" + 0.036*"episode" + 0.023*"series" + 0.012*"season" + 0.011*"soldier" + 0.008*"ship" + 0.008*"military" + 0.007*"american" + 0.007*"pilot" + 0.007*"battle"'),
 (3,
  '0.022*"western" + 0.010*"cartoon" + 0.009*"cagney" + 0.008*"gene" + 0.007*"wayne" + 0.007*"harry" + 0.007*"warner" + 0.007*"flynn" + 0.007*"bogart" + 0.007*"tom"'),
 (4,
  '0.008*"director" + 0.008*"work" + 0.008*"plot" + 0.006*"set" + 0.005*"interesting" + 0.005*"effect" + 0.005*"action" + 0.005*"end" + 0.004*"little" + 0.004*"point"'),
 (5,
  '0.039*"horror" + 0.014*"kill" + 0.012*"killer" + 0.012*"dead" + 0.010*"gore" + 0.010*"monster" + 0.010*"blood" + 0.010*"zombie" + 0

### Visualizing the LDA Results
3 of the circles/topics overlap greatly so it might suggest that the Gensim LDA model did not work as well as the scikit-learn LDA model which used much fewer iterations.  
Although Gensim's model has successfully identified various topics with little overlap, those topics contain terms that account for a smaller amount of the total term frequency, compared to the 3 bigger overlapping circles.

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.224721 -0.068554       1        1  26.507341
1      0.224866 -0.028831       2        1  24.836556
9      0.214677 -0.026185       3        1  21.253527
0     -0.062917 -0.183263       4        1   6.293878
8     -0.032598 -0.213192       5        1   5.196312
5      0.084393  0.222813       6        1   4.865844
7     -0.042771  0.229801       7        1   4.253631
2     -0.117129  0.153624       8        1   2.913967
6     -0.211840 -0.089301       9        1   2.267331
3     -0.281404  0.003087      10        1   1.611612, topic_info=        Term          Freq         Total Category  logprob  loglift
1055  horror   7672.000000   7672.000000  Default  30.0000  30.0000
300     life  15318.000000  15318.000000  Default  29.0000  29.0000
3031  action   8410.000000   8410.000000  Default  28.0000  28.0000
933      war   5188.000000   5188.000000  Default  27.0000  27.0000
432   series   6863.000000   6863.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
2781   lloyd    243.336892    463.954942  Topic10  -5.5965   3.4826
1103   horse    254.224196    792.213906  Topic10  -5.5527   2.9913
436     star    300.127983   8931.512964  Topic10  -5.3867   0.7348
929     town    266.058253   2818.390472  Topic10  -5.5072   1.7677
44      role    281.209003   8492.130023  Topic10  -5.4518   0.7201

[694 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
8772       5  0.991382   aamir
16925      9  0.997186  abbott
2080       3  0.999370   abuse
114        1  0.163695     act
114        2  0.709384     act
...      ...       ...     ...
166        4  0.131677   young
166        5  0.001040   young
166        6  0.014798   young
166        7  0.000462   young
2959       6  0.999620  zombie

[1682 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 10, 1, 9, 6, 8, 3, 7, 4])

##LSA(Latent Semantic Analysis) / LSI(Latent Semantic Indexing)
**How does LSA Work?**  
* A term-document matrix is constructed, where each row represents a term and each column represents a document.
* The values in the matrix represent the frequency of the term in the document.
* The matrix is subjected to SVD, which decomposes it into three matrices: a left singular matrix, a diagonal singular value matrix, and a right singular matrix.
* The resulting matrices capture the underlying relationships between the terms and the documents.
* These matrices can be used to perform various analyses, such as finding the most important topics in the dataset using the left singular matrix.
* The right singular matrix can be used to find the most important terms associated with each topic.

---
**Understanding the Output**  
LSA output values can be positive or negative. The sign of the value reflects the direction of the relationship between the term and the topic.  
A positive value suggests a positive association between the term and the topic, implying that the term is likely to appear in relevant documents.  
Conversely, a negative value indicates a negative association between the term and the topic, implying that the term is more likely to appear in documents that are not relevant to this topic

---

---

##Using Gensim for LSI/LSA


In [ ]:
lsimodel = LsiModel(corpus, id2word=dictionary)
vectorized_corpus = lsimodel[corpus]  # vectorize input copus in BoW format
lsimodel.show_topics(num_topics=10, num_words=10)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.9/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.9/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools`

[(0,
  '0.216*"man" + 0.201*"life" + 0.178*"year" + 0.167*"little" + 0.163*"work" + 0.159*"actor" + 0.156*"plot" + 0.140*"old" + 0.124*"woman" + 0.123*"love"'),
 (1,
  '-0.547*"man" + -0.401*"life" + -0.294*"woman" + 0.258*"plot" + 0.198*"horror" + 0.166*"actor" + 0.149*"action" + -0.133*"young" + -0.124*"love" + 0.112*"original"'),
 (2,
  '-0.575*"man" + 0.527*"life" + -0.201*"woman" + -0.201*"horror" + -0.162*"guy" + 0.148*"actor" + 0.140*"year" + -0.134*"action" + -0.133*"girl" + 0.129*"real"'),
 (3,
  '-0.400*"girl" + 0.367*"actor" + -0.330*"year" + 0.302*"man" + -0.282*"old" + 0.250*"action" + -0.192*"horror" + -0.189*"little" + 0.184*"performance" + 0.184*"role"'),
 (4,
  '-0.413*"life" + 0.382*"actor" + -0.328*"horror" + 0.325*"year" + 0.252*"old" + 0.236*"comedy" + 0.226*"role" + 0.212*"funny" + 0.148*"performance" + -0.127*"game"'),
 (5,
  '-0.396*"year" + -0.356*"game" + 0.326*"girl" + -0.316*"action" + 0.313*"horror" + 0.245*"woman" + 0.213*"actor" + -0.191*"old" + 0.151*"co

### Using a TF-IDF Corpus

TF-IDF (Term Frequency-Inverse Document Frequency) is a statistical measure used to evaluate the importance of a word in a corpus. In a document-term matrix, it calculates the frequency of a term in a document (TF) and weights it by the inverse frequency of the term across all documents (IDF). The higher the TF-IDF score of a term, the more important it is in a document.

LSI Models often benefit from using TF-IDF corpuses instead of a regular TF corpus. Using a TF-IDF corpus in LSI models helps to reduce the impact of high-frequency words that may not be informative, allowing the model to focus more on the underlying topics.

Although TF-IDF weighting can be used in LDA models, it does not have as much impact as in LSI models, since LDA relies more on the frequency of terms within documents and their distribution across topics.

In [ ]:
tfidf_model = TfidfModel(corpus)
tfidf_corpus = tfidf_model[corpus]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lsimodel = LsiModel(tfidf_corpus, id2word=dictionary)
vectorized_corpus = lsimodel[corpus]  # vectorize input copus in BoW format
lsimodel.show_topics(num_topics=10, num_words=10)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.9/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.9/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools`

[(0,
  '0.120*"actor" + 0.114*"plot" + 0.113*"life" + 0.106*"man" + 0.106*"year" + 0.103*"little" + 0.103*"funny" + 0.101*"love" + 0.100*"horror" + 0.096*"work"'),
 (1,
  '0.352*"horror" + -0.193*"life" + 0.179*"funny" + 0.163*"budget" + 0.162*"effect" + -0.152*"love" + 0.142*"terrible" + -0.138*"woman" + 0.134*"low" + -0.133*"family"'),
 (2,
  '0.543*"funny" + 0.468*"comedy" + -0.260*"horror" + 0.232*"laugh" + 0.143*"joke" + -0.142*"book" + -0.112*"effect" + 0.110*"episode" + 0.108*"humor" + 0.101*"hilarious"'),
 (3,
  '0.763*"book" + -0.220*"horror" + 0.189*"episode" + 0.153*"series" + -0.147*"girl" + 0.141*"original" + -0.124*"woman" + 0.121*"version" + 0.117*"novel" + 0.096*"adaptation"'),
 (4,
  '-0.542*"episode" + -0.409*"game" + 0.367*"book" + -0.251*"series" + -0.229*"season" + -0.190*"action" + 0.145*"horror" + 0.136*"girl" + 0.085*"woman" + 0.061*"waste"'),
 (5,
  '-0.397*"horror" + -0.369*"episode" + 0.304*"action" + 0.200*"actor" + -0.186*"book" + -0.175*"girl" + -0.165*"ki

## (Optional) Using Bi-grams to detect Phrases

Adding bi_grams into our corpus for detecting better topics. Bigrams check for words which occur together often, like "New" and "York" and then connects them together with an underscore, like "New_York".

This may not always result in better topics though as you can see below where it mostly detects full names and no coherent topics.

In [ ]:
bigram = Phrases(tokenized_data)
tokenized_data = [bigram[line] for line in tokenized_data]

phrases = Phrases(tokenized_data, min_count=3, threshold=7000)

#Finding the created bi_grams with their scores
phrases_scores = sorted(phrases.find_phrases(tokenized_data).items(), key=lambda x: x[1], reverse=True)
for phrase, score in phrases_scores:
    print(phrase, score)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


guess_kinda_person_find_funny_torturous 240478.693877551
teenage_mutant_ninja_turtles 235669.12
shouf_shouf_habibi 235669.12
chiang_kai_shek 235669.12
dillo_con_parole_mie 235669.12
aaab_baad_khaak 235669.12
willem_van_sande_bakhuyzen 196390.93333333332
drei_tagen_bist_tot 184116.5
bin_ich_schön 184116.5
laberinto_del_fauno 184116.5
johanna_ter_steege 184116.5
polizia_chiede_aiuto 184116.5
todo_sobre_madre 168335.0857142857
coronal_mass_ejection 168335.0857142857
find_funny_torturous_sheer_waste_time 168335.0857142857
dae_jang_geum 163659.1111111111
stink_stink_stink_stink 163659.1111111111
spring_summer_fall_winter_spring 157814.14285714284
vote_title_brazil_mistério 147293.2
lowell_ganz_babaloo_mandel 147293.2
valeria_bruni_tedeschi 147293.2
arturo_pérez_reverte 147293.2
eli_eli_lema_sabachthani 147293.2
star_rating_saturday_night_friday_night_friday_morning 142706.21453287196
seven_deadly_sin_gluttony 140279.23809523808
stop_snitchin_stop_lyin 140279.23809523808
sheer_waste_time_gue

#Conclusion
In this notebook, we explored topic modeling on the IMDB reviews dataset using two popular techniques: LDA and LSA. We experimented with Scikit-learn's LDA model and Gensim's LDA and LSI models, and compared their performance.

Our analysis revealed that Gensim's LDA model was less effective than Scikit-learn's LDA model in identifying distinct and well-separated topics. The visualisation of the Gensim model showed that three of the topics which contained some of the most frequent terms in the corpus had a significant overlap, which suggests that the model did not perform as well as the Scikit-learn model.   
Furthermore, although Gensim's model identified various topics with little overlap, those topics contained terms that accounted for a smaller amount of the total term frequency compared to the three bigger overlapping topics.

Meanwhile Scikit-learn's LDA model produced multiple distinct topics with little overlap and these topics contained terms that were all almost equally frequent in the corpus so it produced good balanced topics.

We also experimented with using a corpus of only adjectives and nouns for the Scikit-learn LDA model. This attempt resulted in even better separation of topics, with the topics overlapping even less than with just the normal corpus. This suggests that using only adjectives and nouns may be a useful preprocessing step for topic modeling on this dataset.

We used Gensim's LSI model as well but the negative values can make it difficult to identify meaningful topics, and LSI is also known to be sensitive to the quality of the input data so it is not as robust as LDA, we can see that it produces very generic topics that have a significant overlap.

Overall, this notebook demonstrates the importance of experimenting with different models and preprocessing techniques to find the most effective approach for topic modeling.
